In [ ]:
import numpy as np

In [ ]:
A1 = np.array([[4, 2],
              [2, -4]])

b1 = np.array([7, -4])

In [ ]:
A2 = np.array([[1, 3, 1],
              [2, 2, 1],
              [3, 1, 2]])

b2 = np.array([8, 9, 13])

In [ ]:
A3 = np.array([[2, -4, 5],
              [3, -3, -5],
              [4, 5, 2]])

b3 = np.array([3, -5, 11])

In [ ]:
np.linalg.solve(A1, b1)

array([1. , 1.5])

In [ ]:
np.linalg.solve(A2, b2)

array([2., 1., 3.])

In [ ]:
np.linalg.solve(A3, b3)

array([1., 1., 1.])

In [ ]:
def solve(A, b, N=20, x=None):

    if x is None:
        x = np.zeros(len(A[0]))

    tau = 2 / np.linalg.norm(A)
    D = np.diagflat(np.diag(A))
    L = np.tril(A, -1)

    for i in range(N):
        x = tau * (np.linalg.inv(D + tau*L) @ (b - A@x)) + x

    return x

In [ ]:
from scipy.linalg import norm as norm
from scipy.sparse import diags

"""The program is to split the matrix into D-diagonal; L: strictly lower matrix; U strictly upper matrix
    satisfying: A = D - L - U  """
def splitMat(A):
    n,m = A.shape
    if (n == m):
        diagval = np.diag(A)
        D = diags(diagval,0).toarray()
        L = (-1)*np.tril(A,-1)
        U = (-1)*np.triu(A,1)
    else:
        print("A needs to be a square matrix")
    return (L,D,U)

"""Preconditioned Matrix for symmetric successive over-relaxation (SSOR): """
def P_SSOR(A,w):
    L,D,U = splitMat(A)
    P = 2/(2-w) * (1/w*D+L)*np.linalg.inv(D)*(1/w*D+L).T
    return P

"""GMRES_SSOR using right preconditioning P:
A - matrix of linear system Ax = b
x0 - initial guess
tol - tolerance
maxit - maximum iteration """

def myGMRES_SSOR(A, b, x0=None, tol=10e-6, maxit=100):


    if x0 is None:
        x0 = np.zeros((len(b),1))

    matrixSize = A.shape[0]
    e = np.zeros((maxit+1,1))
    rr = 1
    rstart = 2
    X = x0
    w = 1.9 ## in ssor
    P = P_SSOR(A,w) ### preconditioned matrix
    ### Starting the GMRES ####
    for rs in range(0,rstart+1):
        ### first check the residual:
        if rr<tol:
            break
        else:
            r0 = (b-A.dot(x0))
            rho = norm(r0)
            e[0] = rho
            H = np.zeros((maxit+1,maxit))
            Qcol = np.zeros((matrixSize, maxit+1))
            Qcol[:,0:1] = r0/rho
        for k in range(1, maxit+1):
            ### Arnodi procedure ##
            Qcol[:,k] =np.matmul(np.matmul(A,P), Qcol[:,k-1])  ### This step applies P here:
            for j in range(0,k):
                H[j,k-1] = np.dot(np.transpose(Qcol[:,k]),Qcol[:,j])
                Qcol[:,k] = Qcol[:,k] - (np.dot(H[j,k-1], Qcol[:,j]))

            H[k,k-1] =norm(Qcol[:,k])
            Qcol[:,k] = Qcol[:,k]/H[k,k-1]

            ###  QR decomposition step ###
            n = k
            Q = np.zeros((n+1, n))
            R = np.zeros((n, n))
            R[0, 0] = norm(H[0:n+2, 0])
            Q[:, 0] = H[0:n+1, 0] / R[0,0]
            for j in range (0, n+1):
                t = H[0:n+1, j-1]
                for i in range (0, j-1):
                    R[i, j-1] = np.dot(Q[:, i], t)
                    t = t - np.dot(R[i, j-1], Q[:, i])
                R[j-1, j-1] = norm(t)
                Q[:, j-1] = t / R[j-1, j-1]

            g = np.dot(np.transpose(Q), e[0:k+1])
            Y = np.dot(np.linalg.inv(R), g)

            Res= e[0:n] - np.dot(H[0:n, 0:n], Y[0:n])
            rr = norm(Res)

            #### second check on the residual ###
            if rr < tol:
                break

        #### Updating the solution with the preconditioned matrix ####
        X = X  + np.matmul(np.matmul(P,Qcol[:, 0:k]), Y) ### This steps applies P here:
    return X

In [ ]:
solve(A1, b1)

array([0.99983796, 1.50079593])

In [ ]:
from scipy.sparse.linalg import bicg

In [ ]:
myGMRES_SSOR(A2, b2.reshape(3,1))

<ipython-input-38-e800085145b8>:76: RuntimeWarning: invalid value encountered in divide
  Q[:, j-1] = t / R[j-1, j-1]


array([[2.],
       [1.],
       [3.]])

In [ ]:
bicg(A3, b3.reshape(3,1))

(array([1., 1., 1.]), 0)